In [1]:

!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1
!pip install torch-geometric ogb
!pip install sentence-transformers transformers
!pip install pandas numpy tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [1]:
import torch
print("Torch version:", torch.__version__)
assert torch.__version__.startswith("2.5"), "This notebook requires PyTorch 2.5.x"


Torch version: 2.5.1+cu124


In [2]:
import os
import json
import pickle
import torch
import pandas as pd
import numpy as np

from ogb.nodeproppred import PygNodePropPredDataset
from torch_geometric.nn import GraphSAGE
from torch_geometric.data import Data

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from tqdm import tqdm
import torch.nn.functional as F


In [3]:
os.makedirs("data/ogbn-arxiv/raw", exist_ok=True)

if not os.path.exists("data/ogbn-arxiv/raw/titleabs.tsv"):
    !wget -q -O data/ogbn-arxiv/raw/titleabs.tsv https://snap.stanford.edu/ogb/data/misc/ogbn_arxiv/titleabs.tsv


In [4]:
dataset = PygNodePropPredDataset(name="ogbn-arxiv", root="data/ogbn-arxiv")
data = dataset[0]

num_nodes = data.num_nodes
num_classes = int(data.y.max().item() + 1)

print(data)

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:08<00:00,  9.67it/s]
Processing...


Extracting data/ogbn-arxiv/arxiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 13797.05it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 2047.00it/s]

Saving...
Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343, 1])



Done!
/usr/local/lib/python3.12/dist-packages/ogb/nodeproppred/dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.pro

In [5]:
split_idx = dataset.get_idx_split()
train_idx = split_idx["train"]
val_idx   = split_idx["valid"]
test_idx  = split_idx["test"]


In [6]:
mapping_file = "data/ogbn-arxiv/ogbn_arxiv/mapping/nodeidx2paperid.csv.gz"
ogb_map = pd.read_csv(mapping_file)
ogb_map.columns = ["ogb_id", "mag_id"]

raw_texts = pd.read_csv(
    "data/ogbn-arxiv/raw/titleabs.tsv",
    sep="\t",
    header=None,
    names=["mag_id", "title", "abstract"],
    on_bad_lines="skip"
)

merged = pd.merge(ogb_map, raw_texts, on="mag_id", how="left")
merged = merged.sort_values("ogb_id")

merged["full_text"] = merged["title"].fillna("") + " " + merged["abstract"].fillna("")
texts = merged["full_text"].tolist()

assert len(texts) == num_nodes
print("Loaded texts:", len(texts))


Loaded texts: 169343


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

data.edge_index = data.edge_index.to(device)
data.y = data.y.squeeze().to(device)


Device: cuda


In [8]:
encoder = SentenceTransformer("all-MiniLM-L6-v2", device=device)
encoder.eval()

emb_path = "arxiv_sbert_embeddings.pt"

if os.path.exists(emb_path):
    node_embeddings = torch.load(emb_path, map_location=device)
else:
    embs = []
    for i in tqdm(range(0, num_nodes, 64)):
        with torch.no_grad():
            e = encoder.encode(texts[i:i+64], convert_to_tensor=True)
        embs.append(e)
    node_embeddings = torch.cat(embs)
    torch.save(node_embeddings.cpu(), emb_path)
    node_embeddings = node_embeddings.to(device)

data.x = node_embeddings
print("Embedding shape:", data.x.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 2646/2646 [02:44<00:00, 16.13it/s]


Embedding shape: torch.Size([169343, 384])


In [9]:
class SAGE(torch.nn.Module):
    def __init__(self, in_dim, hidden, out_dim):
        super().__init__()
        self.net = GraphSAGE(
            in_channels=in_dim,
            hidden_channels=hidden,
            num_layers=2,
            out_channels=out_dim,
            dropout=0.5,
            act="relu"
        )

    def forward(self, x, edge_index):
        return self.net(x, edge_index)

model = SAGE(data.x.size(1), 128, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


In [10]:
train_mask = torch.zeros(num_nodes, dtype=torch.bool, device=device)
val_mask   = torch.zeros(num_nodes, dtype=torch.bool, device=device)
test_mask  = torch.zeros(num_nodes, dtype=torch.bool, device=device)

train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def eval_all():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    return (
        (pred[train_mask] == data.y[train_mask]).float().mean().item(),
        (pred[val_mask] == data.y[val_mask]).float().mean().item(),
        (pred[test_mask] == data.y[test_mask]).float().mean().item(),
    )

for epoch in range(1, 101):
    loss = train()
    tr, va, te = eval_all()
    if epoch % 10 == 0:
        print(f"{epoch:03d} | Loss {loss:.4f} | Train {tr:.4f} | Val {va:.4f} | Test {te:.4f}")


010 | Loss 1.9510 | Train 0.5842 | Val 0.6046 | Test 0.5929
020 | Loss 1.3550 | Train 0.6420 | Val 0.6468 | Test 0.6230
030 | Loss 1.1584 | Train 0.6801 | Val 0.6832 | Test 0.6508
040 | Loss 1.0667 | Train 0.7058 | Val 0.7043 | Test 0.6755
050 | Loss 1.0213 | Train 0.7182 | Val 0.7158 | Test 0.6888
060 | Loss 0.9971 | Train 0.7252 | Val 0.7196 | Test 0.6938
070 | Loss 0.9752 | Train 0.7294 | Val 0.7223 | Test 0.6964
080 | Loss 0.9627 | Train 0.7324 | Val 0.7243 | Test 0.6991
090 | Loss 0.9538 | Train 0.7343 | Val 0.7263 | Test 0.6999
100 | Loss 0.9473 | Train 0.7354 | Val 0.7269 | Test 0.7029


In [11]:
print(
    "Model:", isinstance(model, torch.nn.Module),
    "Texts:", len(texts),
    "Embeddings:", node_embeddings.shape,
    "Test idx:", len(test_idx)
)


Model: True Texts: 169343 Embeddings: torch.Size([169343, 384]) Test idx: 48603


In [12]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    MarianMTModel,
    MarianTokenizer,
)

In [13]:

SUBSET_SIZE = 1000
test_subset = test_idx[:SUBSET_SIZE].cpu()
print("Using test subset size:", len(test_subset))


Using test subset size: 1000


In [14]:

RESULTS_PATH = "paraphrase_attack_results.json"

if os.path.exists(RESULTS_PATH):
    with open(RESULTS_PATH, "r") as f:
        results = json.load(f)
else:
    results = {}

results["test_subset"] = test_subset.tolist()
with open(RESULTS_PATH, "w") as f:
    json.dump(results, f, indent=2)


In [15]:

@torch.no_grad()
def get_preds():
    model.eval()
    return model(data.x, data.edge_index).argmax(dim=1).cpu()

data.x = node_embeddings.clone().to(device)
baseline_preds = get_preds()

baseline_test_acc = (
    baseline_preds[test_subset] == data.y.cpu()[test_subset]
).float().mean().item()

results["baseline"] = {"test_acc": baseline_test_acc}
with open(RESULTS_PATH, "w") as f:
    json.dump(results, f, indent=2)

print("Baseline test accuracy:", baseline_test_acc)


Baseline test accuracy: 0.7170000076293945


In [16]:

def save_results(name, test_acc, flip_rate, cos_dist):
    results[name] = {
        "test_acc": float(test_acc),
        "accuracy_drop": float(baseline_test_acc - test_acc),
        "flip_rate": float(flip_rate),
        "cosine_mean": float(cos_dist.mean().item()),
        "cosine_median": float(cos_dist.median().item()),
        "cosine_max": float(cos_dist.max().item()),
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    }
    with open(RESULTS_PATH, "w") as f:
        json.dump(results, f, indent=2)


In [17]:

PARA_MODEL = "t5-base"
t5_tokenizer = AutoTokenizer.from_pretrained(PARA_MODEL)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(PARA_MODEL, use_safetensors=True).to(device)
t5_model.eval()


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [18]:

@torch.no_grad()
def paraphrase_once(text):
    inputs = t5_tokenizer("paraphrase: "+text, truncation=True, max_length=512, return_tensors="pt").to(device)
    out = t5_model.generate(**inputs, max_length=256, do_sample=True, temperature=1.3, top_p=0.95, num_beams=4)
    return t5_tokenizer.decode(out[0], skip_special_tokens=True)

def paraphrase_two(text):
    return paraphrase_once(paraphrase_once(text))


In [19]:

BT_LANGS = {
    "zh": ("Helsinki-NLP/opus-mt-en-zh", "Helsinki-NLP/opus-mt-zh-en"),
    "hi": ("Helsinki-NLP/opus-mt-en-hi", "Helsinki-NLP/opus-mt-hi-en"),
    "de": ("Helsinki-NLP/opus-mt-en-de", "Helsinki-NLP/opus-mt-de-en"),
}
TRANSLATORS = {}

def load_translator(name):
    tok = MarianTokenizer.from_pretrained(name)
    mdl = MarianMTModel.from_pretrained(
        name,
        weights_only=False
    ).to(device)
    mdl.eval()
    return tok, mdl


for lang,(a,b) in BT_LANGS.items():
    TRANSLATORS[lang] = {"en_x": load_translator(a), "x_en": load_translator(b)}


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/806k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/805k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

target.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [20]:

@torch.no_grad()
def translate(text, tok, mdl):
    inputs = tok(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    out = mdl.generate(**inputs, num_beams=4)
    return tok.decode(out[0], skip_special_tokens=True)

def backtranslate_once(text, lang):
    tok1,mdl1 = TRANSLATORS[lang]["en_x"]
    tok2,mdl2 = TRANSLATORS[lang]["x_en"]
    return translate(translate(text,tok1,mdl1), tok2, mdl2)

def backtranslate_two(text, lang):
    return backtranslate_once(backtranslate_once(text, lang), lang)


In [21]:

def run_attack(name, attack_fn):
    data.x = node_embeddings.clone().to(device)
    attacked = texts.copy()
    for idx in tqdm(test_subset, desc=name):
        attacked[idx] = attack_fn(texts[idx])

    embs=[]
    for i in range(0,len(texts),64):
        with torch.no_grad():
            embs.append(encoder.encode(attacked[i:i+64], convert_to_tensor=True))
    data.x = torch.cat(embs).to(device)

    preds = get_preds()
    test_preds = preds[test_subset]
    base_preds = baseline_preds[test_subset]

    test_acc = (test_preds == data.y.cpu()[test_subset]).float().mean().item()
    flip_rate = (test_preds != base_preds).float().mean().item()

    cos_dist = 1 - F.cosine_similarity(
        node_embeddings[test_subset].cpu(),
        data.x[test_subset].cpu(),
        dim=1
    )
    save_results(name, test_acc, flip_rate, cos_dist)

    torch.save(data.x[test_subset].cpu(), f"embeddings_{name}.pt")
    with torch.no_grad():
        logits = model(data.x, data.edge_index)
        torch.save(logits[test_subset].cpu(), f"logits_{name}.pt")


In [22]:
import time

for lang in ["hi","de"]:
    run_attack(f"bt_{lang}_1step", lambda t,l=lang: backtranslate_once(t,l))


bt_hi_1step:   0%|          | 0/1000 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

bt_de_1step: 100%|██████████| 1000/1000 [59:15<00:00,  3.56s/it]


In [23]:

with open(RESULTS_PATH) as f:
    results=json.load(f)

rows=[]
for k,v in results.items():
    if k in ["baseline","test_subset"]: continue
    rows.append({
        "Attack":k,
        "Test Accuracy":v["test_acc"],
        "Accuracy Drop":v["accuracy_drop"],
        "Flip Rate":v["flip_rate"],
        "Cosine Mean":v["cosine_mean"],
    })
df=pd.DataFrame(rows).sort_values("Accuracy Drop",ascending=False)
df


,Attack,Test Accuracy,Accuracy Drop,Flip Rate,Cosine Mean
4,bt_hi_1step,0.427,0.290,0.501,0.651522
1,paraphrase_2step,0.467,0.244,0.426,0.582928
0,paraphrase_1step,0.562,0.149,0.312,0.394121
3,bt_zh_2step,0.612,0.099,0.272,0.347530
2,bt_zh_1step,0.637,0.074,0.226,0.290863
5,bt_de_1step,0.703,0.014,0.105,0.085570
